# Selenium 구글 이미지 스크래핑

## Selenium 및 웹 드라이버 설치

In [2]:
!pip install Selenium
!apt-get install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome("chromedriver", options= chrome_options)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (84.0.4147.105-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


## 필요 라이브러리 import

In [23]:
import os
import time
import socket

from urllib.request import urlretrieve
from urllib.error import HTTPError, URLError
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException, ElementNotInteractableException
from PIL import Image

## `scroll_down()`: 스크롤을 내리는 함수

In [24]:
def scroll_down():
  scroll_count = 0
  print("[scroll_down(): 스크롤 다운 시작]")

  last_height = wd.execute_script("return document.body.scrollHeight")
  after_click = False

  while True:
    print(f"[스크롤 다운: {scroll_count}]")
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    scroll_count +=1
    time.sleep(1)

    new_height = wd.execute_script("return document.body.scrollHeight")
    if last_height == new_height:
      if after_click is True:
        break
      else:
        try:
          more_button = wd.find_element_by_xpath('//*[@id="islmp"]/div/div/div/div/div[5]/input')
          if more_button.is_displayed():
            more_button.click()
            after_click = True
        except NoSuchElementException as e:
          print(e)
          break

    last_height = new_height



## `click_and_save()`: 썸네일 이미지 선택 후 원본 이미지 저장

In [25]:
def click_and_save(dir_name, index, img, img_list_length):
  global scraped_count
  try:
    img.click()
    wd.implicitly_wait(3)
    src = wd.find_element_by_xpath('//*[@id="Sva75c"]/div/div/div[3]/div[2]/c-wiz/div[1]/div[1]/div/div[2]/a/img').get_attribute('src')
    if src.split('.')[-1] == 'png':
      urlretrieve(src, dir_name + '/' + str(scraped_count + 1) + ".png")
      print(f" {index+1}/{img_list_length} PNG 이미지 저장")
    else:
      urlretrieve(src, dir_name + '/' + str(scraped_count + 1) + ".jpg")
      print(f" {index+1}/{img_list_length} JPG 이미지 저장")
    
    scraped_count +=1
  except HTTPError as e:
    print(e)
    pass
  except ElementClickInterceptedException as e:
    print(e)
    pass


## `scraping()`: 구글 이미지 스크래핑 시작

In [26]:
def scraping(dir_name, query):
  global scraped_count

  url = f"https://www.google.com/search?q={query}&tbm=isch&hl=ko&hl=ko&safe=active&safe=active&tbs=isz%3Al&sa=X&ved=0CAEQpwVqFwoTCLDy9K3XwOsCFQAAAAAdAAAAABAC&biw=1148&bih=1163"
  

  wd.get(url)
  wd.maximize_window()

  scroll_down()

  div = wd.find_element_by_xpath('//*[@id="islrg"]/div[1]')
  img_list = div.find_elements_by_css_selector('div.bRMDJf.islir > img')
  print(img_list)

  for index, img in enumerate(img_list):
    try:
      click_and_save(dir_name, index, img, len(img_list))
    except ElementClickInterceptedException as e:
      print(e)
      wd.execute_script("window.scrollTo(0, window.scrollY + 100")
      time.sleep(1)
      click_and_save(dir_name, index, img, len(img_list))
    except ConnectionError as e:
      print(e)
      pass
    except URLError as e:
      print(e)
      pass
    except socket.timeout as e:
      print(e)
      pass
    except socket.gaierror as e:
      print(e)
      pass
    except ElementNotInteractableException as e:
      print(e)
      break
  try:
    print("[스크래핑 종료 ( 성공률 : %.2f%%)]" % (scraped_count / len(img_list) * 100.0))
  except ZeroDivisionError as e:
    print(e)
  
  wd.quit()

## `filter_and_remove()`: 일정 해상도 이하이거나 손상된 이미지 제거

In [27]:
def filter_and_remove(dir_name, query, filter_size):
  filtered_count = 0

  for index, file_name in enumerate(os.listdir(dir_name)):
    try:
      file_path = os.path.join(dir_name, file_name)
      img = Image.open(file_path)

      if img.width < filter_size and img.height < filter_size:
        img.close()
        os.remove(file_path)
        print(f"{index} 이미지 제거")
        filtered_count +=1
    except OSError as e:
      print(e)
      os.remove(file_path)
      filetered_count +=1

  print(f"[이미지 제거 갯수 : {filtered_count} / {scraped_count}]")

## 이미지 확인

In [29]:
socket.setdefaulttimeout(30)

scraped_count = 0

wd = webdriver.Chrome("chromedriver", options= chrome_options)

path='./'
query = input("검색어 입력: ")

dir_name = path + query
os.makedirs(dir_name)
print(f"[{dir_name} 디렉토리 생성]")

scraping(dir_name, query)
filter_and_remove(dir_name, query, 400)

검색어 입력: forest
[./forest 디렉토리 생성]
[scroll_down(): 스크롤 다운 시작]
[스크롤 다운: 0]
[스크롤 다운: 1]
[스크롤 다운: 2]
[스크롤 다운: 3]
[스크롤 다운: 4]
[스크롤 다운: 5]
[스크롤 다운: 6]
[스크롤 다운: 7]
[스크롤 다운: 8]
[스크롤 다운: 9]
[스크롤 다운: 10]
[스크롤 다운: 11]
[<selenium.webdriver.remote.webelement.WebElement (session="212678e306e639b93e2735de2dc8423a", element="6a50af57-c44e-48cf-9b2f-25eba1e8725c")>, <selenium.webdriver.remote.webelement.WebElement (session="212678e306e639b93e2735de2dc8423a", element="f649ca9d-ec18-4693-acff-ae9dd059fa97")>, <selenium.webdriver.remote.webelement.WebElement (session="212678e306e639b93e2735de2dc8423a", element="46af39f9-c488-4559-90c9-86ea3d0b6cb6")>, <selenium.webdriver.remote.webelement.WebElement (session="212678e306e639b93e2735de2dc8423a", element="3d74250e-e87f-4a0a-855c-b3cfbae6ca2c")>, <selenium.webdriver.remote.webelement.WebElement (session="212678e306e639b93e2735de2dc8423a", element="9b034dff-6272-46b1-a82b-ea5401fa6f50")>, <selenium.webdriver.remote.webelement.WebElement (session="212678e306e63

In [30]:
!ls forest/

100.jpg  176.jpg  265.jpg  357.jpg  442.jpg  519.jpg  585.jpg  691.jpg	812.jpg
105.jpg  177.jpg  269.jpg  360.jpg  443.jpg  520.jpg  591.jpg  712.jpg	821.jpg
106.jpg  180.jpg  278.jpg  365.jpg  448.jpg  523.jpg  592.jpg  714.jpg	822.jpg
110.jpg  181.jpg  282.jpg  367.jpg  451.jpg  524.jpg  598.jpg  717.jpg	827.jpg
11.jpg	 185.jpg  284.jpg  369.jpg  457.jpg  525.jpg  600.jpg  721.jpg	82.jpg
123.jpg  194.jpg  285.jpg  373.jpg  466.jpg  528.jpg  604.jpg  722.jpg	836.jpg
124.jpg  211.jpg  292.jpg  376.jpg  46.jpg   535.jpg  608.jpg  724.jpg	839.jpg
135.jpg  213.jpg  2.jpg    378.jpg  471.jpg  540.jpg  613.jpg  743.jpg	840.jpg
139.jpg  217.jpg  30.jpg   379.jpg  474.jpg  541.jpg  616.jpg  744.jpg	842.jpg
142.jpg  218.jpg  311.jpg  381.jpg  475.jpg  543.jpg  622.jpg  745.jpg	843.jpg
146.jpg  224.jpg  319.jpg  384.jpg  476.jpg  545.jpg  626.jpg  749.jpg	845.jpg
147.jpg  237.jpg  321.jpg  389.jpg  47.jpg   54.jpg   631.jpg  750.jpg	846.jpg
14.jpg	 242.jpg  32.jpg   397.jpg  480.jpg  555.jpg  6

In [31]:
!rm -rf forest